# A "real life" example

In [ ]:
import os
from jobqueue_features import (
    MPIEXEC,
    CustomSLURMCluster,
)
from jobqueue_features import on_cluster, mpi_task, get_task_mpi_comm
from jobqueue_features.clusters_controller import (
    clusters_controller_singleton as controller,
)

In this case, let's allow each worker to use one node. Since we have 2
available nodes, we can have a maximum of 2 running jobs

In [ ]:
custom_cluster = CustomSLURMCluster(
    name="mpiMultiCluster",
    nodes=1,
    maximum_jobs=2,
    mpi_mode=True
)

Instead of using `cluster` to identify the cluster to the decorator we can also
use a string argument `cluster_id` to identify the cluster we need. This can be a useful
alternative to passing around the cluster instance.

In [ ]:
@mpi_task(cluster_id=custom_cluster.name)
def lammps_task(input_file, run_steps=100):
    from mpi4py import MPI
    from lammps import PyLammps
    
    L = PyLammps()      # Initialise LAMMPS
    L.file(input_file)  # Read the input file
    L.run(run_steps)    # Simulate the system

    if MPI.COMM_WORLD.Get_rank()==0:
        return "Potential energy: %s" % L.eval("pe")

Now let's execute this task with an available default input file
and a default of 100 steps

In [ ]:
@on_cluster(cluster=custom_cluster)
def my_lammps_job(input_file="in.melt", run_steps=100):
    return lammps_task(input_file, run_steps=run_steps)

In [ ]:
future = my_lammps_job(run_steps=2000)
print(future.result())
future.cancel()

In [ ]:
controller._close()

## Computing environments
But there should have been a problem with these tasks. It turns out that the `lammps` module is not available:

In [ ]:
import lammps

Why didn't this trigger an error? This is because the computing environment on the nodes can be completely different to the one where our notebook is running. On the nodes `c1` and `c2` the LAMMPS python module is indeed available.

On HPC systems it is frequently the case that you need to configure a very
specific computing environment for your tasks. The package LAMMPS may come as an
*environment module* that you need to load.

We can handle the creation of our computing environment when we declare our
cluster, for example,
```
GROMACS_gpu_cluster = CustomSLURMCluster(
    name="GROMACS_gpu_cluster",
    nodes=2,
    mpi_mode=True,
    fork_mpi=True,
    queue_type="gpus",
    maximum_jobs=5,
    job_extra_directives=[
        "module load Intel",
        "module load IntelMPI",
        "module load GROMACS",
        "module load dask",
        "module load jobqueue_features",
    ],
)
```

In this way we can declare different clusters for different task workloads and have them interact